In [1]:
import time
import numpy as np
import io
import os
import torch.nn.functional as F

import torch
from PIL import Image
import cv2
import saverloader
from nets.pips import Pips
import imageio.v3 as imageio
from demo import run_model
import utils.improc
import random
import glob
from utils.basic import print_, print_stats
from tensorboardX import SummaryWriter

In [2]:
file = open('/media/shirbar/DATA/codes/pips/feeding.avi')
file.close()
vid=imageio.imread('/media/shirbar/DATA/codes/pips/feeding.avi',plugin='pyav')

deprecated pixel format used, make sure you did set range correctly


In [3]:
vid.shape

(41, 400, 400, 3)

In [4]:
tens_vid = torch.from_numpy(vid).permute(0,3,1,2)
tens_vid = tens_vid.unsqueeze(dim=0)
tens_vid.shape

torch.Size([1, 41, 3, 400, 400])

In [5]:
tens_vid[:,0:40:5,...].shape

torch.Size([1, 8, 3, 400, 400])

In [5]:
model = Pips(stride=4).cuda()
init_dir = 'reference_model'
_ = saverloader.load(init_dir, model)
model.eval()

reading ckpt from reference_model
...found checkpoint reference_model/model-000200000.pth


Pips(
  (fnet): BasicEncoder(
    (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (norm2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      

In [7]:
B = 1
S = 8
N = 16**2
log_freq = 2
global_step = 0
log_dir = './logs_demo'
model_name = 'slkjgdlk'
writer_t = SummaryWriter(log_dir + '/' + model_name + '/t', max_queue=10, flush_secs=60)

sw_t = utils.improc.Summ_writer(
    writer=writer_t,
    global_step=global_step,
    log_freq=log_freq,
    fps=5,
    scalar_freq=int(log_freq/2),
    just_gif=True)

In [11]:
#run_model(model,tens_vid[:,0:8,...],N,sw_t)
rgbs = tens_vid[:,0:40:5,...].cuda().float()
B, S, C, H, W = rgbs.shape
rgbs_ = rgbs.reshape(B*S, C, H, W)
H_, W_ = 360, 640
rgbs_ = F.interpolate(rgbs_, (H_, W_), mode='bilinear')
H, W = H_, W_
rgbs = rgbs_.reshape(B, S, C, H, W)

In [8]:
rgbs = tens_vid[:,0:40:5,...].cuda().float()

B, S, C, H, W = rgbs.shape
N_ = np.sqrt(N).round().astype(np.int32)
grid_y, grid_x = utils.basic.meshgrid2d(B, N_, N_, stack=False, norm=False, device='cuda')
grid_y = 8 + grid_y.reshape(B, -1)/float(N_-1) * (H-16)
grid_x = 8 + grid_x.reshape(B, -1)/float(N_-1) * (W-16)
xy = torch.stack([grid_x, grid_y], dim=-1) # B, N_*N_, 2
_, S, C, H, W = rgbs.shape
preds, preds_anim, vis_e, stats = model(xy, rgbs, iters=6)

In [9]:
trajs_e = preds[-1]
print_stats('trajs_e', trajs_e)

pad = 50
rgbs = F.pad(rgbs.reshape(B*S, 3, H, W), (pad, pad, pad, pad), 'constant', 0).reshape(B, S, 3, H+pad*2, W+pad*2)
trajs_e = trajs_e + pad

trajs_e (float32) min = 3.77, mean = 202.45, max = 394.69 torch.Size([1, 8, 256, 2])


In [10]:
trajs_e.shape

torch.Size([1, 8, 256, 2])

In [11]:
linewidth=2
o1 = sw_t.summ_rgbs('inputs/rgbs', utils.improc.preprocess_color(rgbs[0:1]).unbind(1))
o2 = sw_t.summ_traj2ds_on_rgbs('outputs/trajs_on_rgbs', trajs_e[0:1], utils.improc.preprocess_color(rgbs[0:1]), cmap='spring', linewidth=linewidth)
o3 = sw_t.summ_traj2ds_on_rgbs('outputs/trajs_on_black', trajs_e[0:1], torch.ones_like(rgbs[0:1])*-0.5, cmap='spring', linewidth=linewidth)


In [12]:
o3.shape

torch.Size([1, 8, 3, 500, 500])

In [13]:
wide_cat = torch.cat([o1, o2, o3], dim=-1)
sw_t.summ_rgbs('outputs/wide_cat', wide_cat.unbind(1))

tensor([[[[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],

          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],

          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]],


         [[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],

          [[0, 0, 0,  ..., 0, 0, 0],
 

In [14]:
wide_list = list(wide_cat.unbind(1))
wide_list = [wide[0].permute(1,2,0).cpu().numpy() for wide in wide_list]
wide_list = [Image.fromarray(wide) for wide in wide_list]
out_fn = './out_%d.gif' % sw_t.global_step
wide_list[0].save(out_fn, save_all=True, append_images=wide_list[1:])
print('saved %s' % out_fn)


saved ./out_0.gif
